Aprendiendo Simpy

Lo primero es entender que es Simpy.

Sympy es una libreria de python de simulación de eventos discretos. 

Todo lo que sucede tiene lugar en un 'enviroment' o 'entorno', este entorno responderá a eventos que rocurriran a lo largo del tiempo. Note que dichos eventos ocurren en tiempo discreto. Para crear dicho entorno, es necesario usar el comando *Enviroment()* y asignarlo a una variable. Se recomienda que dicha variable sea 'env', 'ent' o 'entorno' para facilitar su comprensión y uso.


Se pueden hacer eventos a travéz de funciones, métodos o clases, a los cuales llamaremos 'procesos' o 'process'. Estos eventos tienen un inicio y un final. Un process es la cosa o accion que se realiza mientras que el tiempo transcurre hasta el un evento modifica el entorno. En otras palabras, un proceso es un intervalo de tiempo definido entre dos eventos.


Una vez que el proceso ha terminado (para lo cual utilizamos el comando *yield*), momento en que ocurre un evento, simpy entrega un resumen del proceso. Esto se puede denotar con la frase 'event is processed'. Multiples procesos (acciones, cosas, situaciones, etc.) pueden ocurrir a la vez, simpy las recopila en el mismo orden en que finalizaron (yielded).

Una manera de crear un evento es a travéz del comando "timeout". Los eventos de este tipo ocurren tras un lapso de tiempo, lo cual detiene el proceso (sleep). Su sintaxis es *Enviromet.timeout()*, donde 'Enviroment' es la respectiva variable de tipo *simpy.core.Environment*, es decir, nuestra variable 'env', 'ent' o 'entorno'. De modo que la sintaxis para los eventos es algo parecido a *env.timeout(time)*. Notese que time es el valor numérico que representa el lapso de tiempo transcurrido ($\Delta t$).


Ahora bien, para iniciar vamos a crear un proceso que simule el estado de un carro, el cual va a variar entre conducción y aparcado.

In [1]:
def carro(env): #Se usa la variable de entrada env como referencia del "enviroment" o "entorno"
    while True: 
        print(f'Inicia el estacionado en: {env.now}') #env.now regresa el tiempo actual de la simulación
        duracion_aparcado = 5
        yield env.timeout(duracion_aparcado) #se crea el evento de estacionado/aparcado

        print(f'Inicia la conducción en: {env.now}') 
        duracion_conduccion = 2
        yield env.timeout(duracion_conduccion) #se crea el evento de conducción
        

Nuestra funcion de proceso requiere la entrada env para crear los eventos en el transcurso del tiempo. Dentro del ciclo while (aparentemente infinito) se encuentra la simulacion de la conducta del carro, el cual para nuestro ejemplo puede tener dos estados posibles: estacionado (o aparcado) y conduciendo. Como se habrá percatado, parece haber un bucle sin fin, pero este terminará cuando se llegue al tiempo especificado por *run(until=)*, es decir, hasta que el tiempo de ejecución termine. Una vez terminada la simulación, se muestra un resumen de esta.

Ahora que hemos creado nuestro proceso de carro, a continucación vamos a generar uno (crear una instancia) para ver como se comporta:

In [2]:
import simpy

env = simpy.Environment()
env.process(carro(env))

<Process(carro) object at 0x1d7f52eb190>

Y se ejecuta.

In [3]:
env.run(until=24)

Inicia el estacionado en: 0
Inicia la conducción en: 5
Inicia el estacionado en: 7
Inicia la conducción en: 12
Inicia el estacionado en: 14
Inicia la conducción en: 19
Inicia el estacionado en: 21


Como podemos observar, nos imprime el momento en que sucede cada evento.

La instancia de "process" que resulta del comando "Enviroment.process()" se puede sar para interacciones de procesos, como por ejemplo la secuencia de procesos y la interupción de procesos.

***Secuencia de procesos***

Tambien conocido como *Waiting for a Process*, se pueden utilizar cuando tenemos eventos seguidos uno tras de otro, en secuencia. Un  ejemplo de esto es el anteriormente hecho.

***Interrupción de procesos***

Juan trabaja en un despacho jurídico 8 horas al día 5 días a la semana, su entrada es a las 8 diariamente. Juan pidió permiso de salir antes porque el martes tiene que ir a la junta de la escuela de su hija a las 12pm (medio día), por lo que no va a regresar a trabajar sinó hasta el día sguiente. La junta de la escuela durará 2 horas.

Vamos a simular la semana de Juan, de Lunes a Viernes.

In [4]:
class Juan(object):
    def __init__(self, env):
        self.env = env
        self.accion = env.process(self.rutina())
    
    def rutina(self):
        semana = {0:'Lunes', 1:'Martes', 2:'Miercoles', 3:'Jueves', 4:'Viernes'}
        while True:
            dia = self.env.now//24
            print(f'Día {semana[dia]}')
            #Antes del trabajo
            
            hora = self.env.now%24
            print(f'A las {hora} horas Juan esta en su casa')
            yield self.env.timeout(8)

            #Llega al trabajo
            hora = self.env.now%24
            print(f'A las {hora} horas Juan esta en su trabajo')

            try:
                #Trabaja sus 8 horas
                yield self.env.process(self.trabajar(8))
            except simpy.Interrupt:
                #Se tiene que ir a la junta de su hija
                dia = self.env.now//24
                hora = self.env.now%24
                print(f'A las {hora} horas Juan va a la junta de su hija')
                yield self.env.timeout(2)
            
            #Llega a la casa
            hora = self.env.now%24
            print(f'A las {hora} horas Juan esta en su casa\n')
            yield self.env.timeout(24-hora)


    def trabajar(self, duracion):
            yield self.env.timeout(duracion)

In [5]:
def junta(env, persona):
    yield env.timeout(24+12)
    persona.accion.interrupt()

In [6]:
env = simpy.Environment()
persona = Juan(env)
env.process(junta(env,persona))
env.run(until=120)

Día Lunes
A las 0 horas Juan esta en su casa
A las 8 horas Juan esta en su trabajo
A las 16 horas Juan esta en su casa

Día Martes
A las 0 horas Juan esta en su casa
A las 8 horas Juan esta en su trabajo
A las 12 horas Juan va a la junta de su hija
A las 14 horas Juan esta en su casa

Día Miercoles
A las 0 horas Juan esta en su casa
A las 8 horas Juan esta en su trabajo
A las 16 horas Juan esta en su casa

Día Jueves
A las 0 horas Juan esta en su casa
A las 8 horas Juan esta en su trabajo
A las 16 horas Juan esta en su casa

Día Viernes
A las 0 horas Juan esta en su casa
A las 8 horas Juan esta en su trabajo
A las 16 horas Juan esta en su casa



***Recursos compartidos***

Simpy tine el tipo *resources* (recurso) para cuando multiples procesos requieren de un mismo recurso. Esto lo podemos ver cuando hay dos procesos y uno no puede empezar hasta que el otro termine, como por ejemplo los modelos de servidores con disciplina FIFO (PEPS en español).

Para ello usaremos de ejemplo una gasolinerade una sola bomba. Cuando lleguen dos carros consecutivos será necesario que el segundo espere hasta que el primero sea atendido y se marche de la bomba.

Cuando se produzca el evento de la llegada de un carro a la gasolinería, este hará la *solicitud* para cargar gasolina. Si la bomba ya está en uso, esperará hasta que esta se desocupe, en ese momento comenzará la carga de su gasolina.

El método *request()* genera un evento que permite esperar hasta que el recurso este disponible. Para nuestro ejemplo, es lo que indica que el segundo carro se mantenga en espera hasta que primero termine de cargar gasolina.

Y hay dos maneras de generar un request o solisitud: con *with* o con *release()*. El primero hace que el carro ocupe el lugar de manera automática cuando se desocupa la bomba. Mientras que el segundo sería esperar hasta que de la señal para ocupar la bomba, es decir, de manera manual. Para nuestro ejemplo nos combiene usar *with*.

In [7]:
import simpy
from random import random as rd
from math import log

In [8]:
def gasolinera(env, bomba, num, ll, ls):
    for i in range(num): #El for es para delimitar en cuanto al número de carros que se van a atender
        llegada = -log(rd())/ll #Se calcula el tiempo de llegada del carro
        yield env.timeout(llegada)
        print(f'---> Ha llegado el carro {i} en el minuto {round(env.now, 2)}')
        env.process(carro(env, bomba, i, ls))

In [9]:
def carro(env, bomba, num, ls): #esta funcion modela el servicio del carro, es decir, la carga
    #Solicitud de carga de gasolina
    with bomba.request() as req:
        yield req

        #Se carga gasolina
        print(f'El carro {num} se comienza a cargar en el minuto {round(env.now, 2)}')
        carga = -log(rd())/ls #Se calcula el tiempo de carga (servicio) 
        yield env.timeout(carga)
        print(f'El carro {num} termina de cargar en el minuto {round(env.now, 2)}   --->')

In [10]:
entorno = simpy.Environment()

In [11]:
bomba = simpy.Resource(entorno, capacity=1)

In [12]:
entorno.process(gasolinera(entorno, bomba, 7, 1/2, 1/3))

<Process(gasolinera) object at 0x1d7f65fd090>

In [13]:
entorno.run()

---> Ha llegado el carro 0 en el minuto 0.33
El carro 0 se comienza a cargar en el minuto 0.33
El carro 0 termina de cargar en el minuto 1.57   --->
---> Ha llegado el carro 1 en el minuto 4.28
El carro 1 se comienza a cargar en el minuto 4.28
El carro 1 termina de cargar en el minuto 6.25   --->
---> Ha llegado el carro 2 en el minuto 6.7
El carro 2 se comienza a cargar en el minuto 6.7
---> Ha llegado el carro 3 en el minuto 6.81
---> Ha llegado el carro 4 en el minuto 6.91
---> Ha llegado el carro 5 en el minuto 7.67
El carro 2 termina de cargar en el minuto 8.65   --->
El carro 3 se comienza a cargar en el minuto 8.65
---> Ha llegado el carro 6 en el minuto 9.9
El carro 3 termina de cargar en el minuto 10.9   --->
El carro 4 se comienza a cargar en el minuto 10.9
El carro 4 termina de cargar en el minuto 10.91   --->
El carro 5 se comienza a cargar en el minuto 10.91
El carro 5 termina de cargar en el minuto 11.25   --->
El carro 6 se comienza a cargar en el minuto 11.25
El carro 6